In [47]:
"""

This code applies a transformation given by a file labelled "y_filename" onto file "filename" with trilinear interpolation.
I think this is resampling/normalization. The transformation from y_filename moves base coords to the template space, which I 
think is based off the montreal neurological institute (mni) system.

Theo Chiang

"""





import os
import nibabel as nib
import numpy as np
import SimpleITK as sitk
import matlab.engine
import math as math
import matplotlib.pyplot as plt
TINY = 5e-2

def_path='/Users/theochiang/Documents/MATLAB/nifti_files/y_FLAIR<=T2_Flair_Axial_p2_5.nii';
#Transformation Field


img_path='/Users/theochiang/Documents/MATLAB/nifti_files/FLAIR<=T2_Flair_Axial_p2_5.nii';
#base image




#1 - SETUP
#transformation data and affine
trans_img = nib.load(def_path)
trans_data = trans_img.get_fdata()
trans_affine = trans_img.affine


#base image data and affine
base_img = nib.load(img_path)
base_data = base_img.get_fdata()
base_data = base_data.astype(np.float32)

base_affine = base_img.affine

base_data_sitk = sitk.GetImageFromArray(base_data)#(base_data.astype(np.float64), isVector=True)




#2 - SOME ORGANIZATION
#d variable = size/dims of previous variable
trans_data_dims = trans_data.shape

trans_data = trans_data.reshape((121, 145, 121, 3))
trans_data = trans_data.astype(np.float32)

#single precision for transformation field data (not affine)
trans_data_sitk = sitk.GetImageFromArray(trans_data.astype(np.float32), isVector=True)

print(base_data_sitk.GetDimension())
print(trans_data_sitk.GetDimension())

#sITK IMAGE CREATED FOR TRANSFORMATION FIELD IDK IF WE NEED IT?

#AFFINE FUNCTION
def affine(y, M):
    """
    Perform affine transformation on the input data y using the transformation matrix M.
    
    Parameters:
    y (numpy.ndarray): Input data, expected to be a 4D array where the last dimension has size 3.
    M (numpy.ndarray): Transformation matrix, expected to be a 4x4 matrix.
    
    Returns:
    numpy.ndarray: Transformed data, same shape as input data y.
    """
    # Initialize the output array with zeros of the same shape as y and of type float32
    Def = np.zeros(y.shape, dtype=np.float32)
    
    # Apply the affine transformation w/ matrix multiplication affine * transformation field
    Def[..., 0] = y[..., 0] * M[0, 0] + y[..., 1] * M[0, 1] + y[..., 2] * M[0, 2] + M[0, 3] + 1
    Def[..., 1] = y[..., 0] * M[1, 0] + y[..., 1] * M[1, 1] + y[..., 2] * M[1, 2] + M[1, 3] + 1
    Def[..., 2] = y[..., 0] * M[2, 0] + y[..., 1] * M[2, 1] + y[..., 2] * M[2, 2] + M[2, 3] + 1
    
    return Def




#3 - Matrix to get from physical to image?
M = np.linalg.inv(base_img.affine)
M = M.astype(np.float32)

#Now Def is the matrix that converts any physical coords to warped/new image coords
Y = affine(trans_data,M)


#this is not really used:
Y_sitk = sitk.GetImageFromArray(Y.astype(np.float32), isVector=True)
Y_sitk.GetSize()




3
3


(121, 145, 121)

In [48]:
#4 - TRYING TO THE interpolation step ("NONRIGID, RESAMPLING?")

""" 
print(Y_sitk.GetPixelIDTypeAsString())  # Check the pixel type
print(Y_sitk.GetDimension())    #dimension Y
print(Y_sitk.GetSize())         #Size Y
print(base_data_sitk.GetSize()) #base image Y
"""


#mirror edges
def mirror(i, m):
    # i = coord, m is length of box
    i = abs(i)
    if i < m:
        return i
    if m == 1:
        return 0
    m2 = (m - 1) * 2
    i %= m2
    return i if i < m else m2 - i

# SECOND FUNCTION FROM MATLAB
#Accounting for the -1 input (in C++, input x[j]-1, y[j]-1, z[j]-1 for defxyz)
def resample3python(f0, f0_dimx,f0_dimy,f0_dimz, def_field):
    # Transformation field/deformation field x,y,z
    x = def_field[0] - 1 
    y = def_field[1] - 1
    z = def_field[2] - 1
    
    if ((((x+1) >= 1 - TINY and (x+1) <= f0_dimx + TINY)) and
            (((y+1) >= 1 - TINY and (y+1) <= f0_dimy + TINY)) and
            (((z+1) >= 1 - TINY and (z+1) <= f0_dimz + TINY))):

        # Weights
        floored_x = math.floor(x)
        floored_y = math.floor(y)
        floored_z = math.floor(z)

        weights_x = [(1-(x - floored_x)),(x - floored_x)]
        weights_y = [(1-(y - floored_y)),(y - floored_y)]
        weights_z = [(1-(z - floored_z)),(z - floored_z)]

        # Edge control - mirroring def coords
        # Process happens for coord and coord + 1 (so really def_fieldxyz-1,def_field)


        transformation_xs = [mirror(floored_x, f0_dimx), mirror(floored_x + 1, f0_dimx)]
        transformation_ys = [mirror(floored_y, f0_dimy), mirror(floored_y + 1, f0_dimy)]
        transformation_zs = [mirror(floored_z, f0_dimz), mirror(floored_z + 1, f0_dimz)]

        # Convolution

        d2 = 0.0
        for i2 in range(len(weights_z)):
            d1 = 0.0
            for i1 in range(len(weights_y)):
                d0 = 0.0
                for i0 in range(len(weights_x)):
                    d0 += f0[transformation_xs[i0]][transformation_ys[i1]][transformation_zs[i2]] * weights_x[i0]
                d1 += d0 * weights_y[i1]
            d2 += d1 * weights_z[i2]
        #for every Y vector value, the int part of x is the nth image in that dim in base image that correlates to new 
        #pixel being editted. decimal is the influence of that pixel vs the one just below. basically, the kernel is 2x2x2, 
        #and the transformation value specifies the weights, that add up to one between x and x-1.


        return d2
    else:
        return np.nan



#          CALLING THE FUNCTION:


transformed_data = np.empty(trans_data.shape[:-1])
#iterate through x,y,z of transformation/deformation field
for t0 in range( trans_data.shape[0] ):
    for t1 in range( trans_data.shape[1] ):
        for t2 in range( trans_data.shape[2] ):
            
            transformed_data[t0][t1][t2] = resample3python(base_data,base_data.shape[0],base_data.shape[1],base_data.shape[2],Y[t0][t1][t2])
           


In [49]:



# 5- PROCESS OF CREATING NEW FILE
resampled_image = nib.Nifti1Image(transformed_data, trans_img.affine)

def check_if_file_exists(folder_path, filename):
    file_path = os.path.join(folder_path, filename)
    return os.path.exists(file_path)

num = 0
filename = 'nun'
def checkFile():
    global num, filename
    #print(check_if_file_exists('/Users/theochiang/Documents/MATLAB/nifti_files',f"{num}transformed_FLAIR<=T2_Flair_Axial_p2_5.nii"))
    if check_if_file_exists('/Users/theochiang/Documents/MATLAB/nifti_files',f"{num}transformed_FLAIR<=T2_Flair_Axial_p2_5.nii"):
        #print("added one to " + str(num))
        num += 1
        checkFile()
    else:
        filename = f"{num}transformed_FLAIR<=T2_Flair_Axial_p2_5.nii"
checkFile()

output_dir = '/Users/theochiang/Documents/MATLAB/nifti_files'
output_path = os.path.join(output_dir,filename)

nib.save(resampled_image,output_path)

print(f"Transformed image saved to {output_path}")

Transformed image saved to /Users/theochiang/Documents/MATLAB/nifti_files/8transformed_FLAIR<=T2_Flair_Axial_p2_5.nii


In [44]:
base_data_sitk = sitk.GetImageFromArray(base_data,isVector=False)
trans_data_sitk = sitk.GetImageFromArray(trans_data, isVector=False)
transformed_data_sitk = sitk.GetImageFromArray(transformed_data,isVector=False)

print(base_data_sitk.GetSize())
print(trans_data_sitk.GetSize())
print(transformed_data_sitk.GetSize())

(28, 256, 232)
(3, 121, 145, 121)
(121, 145, 121)


In [45]:
base_data_sitk = sitk.GetImageFromArray(base_data.astype(np.float32), isVector=True)
trans_data_sitk = sitk.GetImageFromArray(trans_data.astype(np.float32), isVector=True)
transformed_data_sitk = sitk.GetImageFromArray(transformed_data.astype(np.float32), isVector=True)

print(base_data_sitk.GetSize())
print(trans_data_sitk.GetSize())
print(transformed_data_sitk.GetSize())

(256, 232)
(121, 145, 121)
(145, 121)


In [46]:
print(base_data.shape)
print(trans_data.shape)
print(transformed_data.shape)

(232, 256, 28)
(121, 145, 121, 3)
(121, 145, 121)
